## Yelp Project Part I: Data Preprocessing

### 1. Transform original Business Json data sets to csv

In [1]:
import pandas as pd  
import numpy as np
import json  
import ast
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings("ignore")

In [2]:
yelp_business = []
for line in open('business.json', encoding="utf8"):
    yelp_business.append(json.loads(line))

In [3]:
# Deal witht the embedded Json structure
data = dict()
json_file = []
def format_dict(var, str_key):
    global data
    try: 
        var = ast.literal_eval(var)
    except: pass
    
    if type(var) == type({}):
        for key in var:
            tmp_key = str_key + "_" + key
            format_dict(var[key], tmp_key)    
    else:
        #print({str_key[2:] : var})
        data = {**data, **{str_key[2:] : var}}
        return 0


In [4]:
for i in yelp_business:
    format_dict(i, " ")
    json_file.append(data)

In [5]:
df = json_normalize(json_file)

In [8]:
df.shape

(192609, 109)

In [9]:
IsNa = df.isnull().sum()
agg_category = list(IsNa[IsNa == len(df)].index)

In [10]:
agg_category

['attributes',
 'attributes_Ambience',
 'attributes_BestNights',
 'attributes_BusinessParking',
 'attributes_DietaryRestrictions',
 'attributes_GoodForMeal',
 'attributes_HairSpecializesIn',
 'attributes_Music',
 'hours']

In [11]:
df.drop(agg_category,axis=1,inplace=True)

In [12]:
df.columns

Index(['address', 'attributes_AcceptsInsurance', 'attributes_AgesAllowed',
       'attributes_Alcohol', 'attributes_Ambience_casual',
       'attributes_Ambience_classy', 'attributes_Ambience_divey',
       'attributes_Ambience_hipster', 'attributes_Ambience_intimate',
       'attributes_Ambience_romantic', 'attributes_Ambience_touristy',
       'attributes_Ambience_trendy', 'attributes_Ambience_upscale',
       'attributes_BYOB', 'attributes_BYOBCorkage',
       'attributes_BestNights_friday', 'attributes_BestNights_monday',
       'attributes_BestNights_saturday', 'attributes_BestNights_sunday',
       'attributes_BestNights_thursday', 'attributes_BestNights_tuesday',
       'attributes_BestNights_wednesday', 'attributes_BikeParking',
       'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards',
       'attributes_BusinessParking_garage', 'attributes_BusinessParking_lot',
       'attributes_BusinessParking_street', 'attributes_BusinessParking_valet',
   

In [13]:
# select all the restaurant subsets from the origin big file
restaurant_index = ['Restaurants' in i for i in df['categories'].astype(str)]

In [14]:
df = df.iloc[restaurant_index, :]

In [15]:
df.shape

(59371, 100)

In [18]:
df.head()

,address,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience_casual,attributes_Ambience_classy,attributes_Ambience_divey,attributes_Ambience_hipster,attributes_Ambience_intimate,attributes_Ambience_romantic,...,hours_Tuesday,hours_Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,NaN,NaN,full_bar,True,False,False,False,False,False,...,9:0-0:0,9:0-0:0,1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15",NaN,NaN,beer_and_wine,True,False,False,False,False,False,...,9:0-0:0,17:30-21:30,1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,False,NaN,none,False,False,False,False,False,False,...,7:0-0:0,7:0-0:0,1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,False,NaN,none,False,False,False,False,False,False,...,10:0-0:0,10:0-0:0,1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29",False,NaN,full_bar,False,True,False,False,False,True,...,9:0-19:0,9:0-17:0,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV


In [20]:
df.to_csv('Business_restaurant_orgin.csv', index = False)

### 2. Process Restaurant subsets

In [102]:
df = pd.read_csv('Business_restaurant_orgin.csv')

#### Process operating hours

In [103]:
df.iloc[:,85:92].head(2) # operating hours columns

,hours_Friday,hours_Monday,hours_Saturday,hours_Sunday,hours_Thursday,hours_Tuesday,hours_Wednesday
0,9:0-1:0,9:0-0:0,9:0-1:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0
1,17:30-22:0,17:30-21:30,17:30-22:0,17:30-21:0,17:30-21:30,9:0-0:0,17:30-21:30


In [104]:
# since operating hours columns are time string, we processed the operating hours columns to 
# length of each operating days and whether they include the breakfast, brunch, lunch, dinner
# and night meals. 

In [105]:
df=df.iloc[:,85:92]
df['hours_Friday'][1].split("-")
def timesplit(data,col):
    a=[]
    for i in range(data.shape[0]):
       a+=data[col][i].split('-')
    b=[]
    for ii in range(len(a)):
        b+=a[ii].split(':')
    return b

fri=timesplit(df,'hours_Friday')
mon=timesplit(df,'hours_Monday')
tue=timesplit(df,'hours_Tuesday')
wed=timesplit(df,'hours_Wednesday')
thu=timesplit(df,'hours_Thursday')
sat=timesplit(df,'hours_Saturday')
sun=timesplit(df,'hours_Sunday')

#identify meals served
#breakfast
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i] <= 8 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Breakfast']=breakfast

#brunch
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i] <= 10 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Brunch']=breakfast

#lunch
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i] <= 12 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Lunch']=breakfast

#dinner
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i] <= 17 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Dinner']=breakfast

#earlynight
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i] <= 21 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Earlynight']=breakfast

#latenight
def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        if open[i]<= 25 <= close[i]:
            serve+=[1]
        else:
            serve+=[0]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

breakfast=[]
for i in range(len(fri1)):
    breakfast+=[fri1[i]+mon1[i]+tue1[i]+wed1[i]+thu1[i]+sat1[i]+sun1[i]]
    if breakfast[i]>=4:
        breakfast[i]=1
    else:
        breakfast[i]=0

df['Latenight']=breakfast

In [106]:
# calculate opening hours
def timesplit(data,col):
    a=[]
    for i in range(data.shape[0]):
       a+=data[col][i].split('-')
    b=[]
    for ii in range(len(a)):
        b+=a[ii].split(':')
    return b

fri=timesplit(df,'hours_Friday')
mon=timesplit(df,'hours_Monday')
tue=timesplit(df,'hours_Tuesday')
wed=timesplit(df,'hours_Wednesday')
thu=timesplit(df,'hours_Thursday')
sat=timesplit(df,'hours_Saturday')
sun=timesplit(df,'hours_Sunday')

def serve(day):
    open=[]
    close=[]
    serve=[]
    for i in range(len(day)):
        if i%4 == 0:
            open+=[int(day[i])]
        elif i%2 == 0:
            close+=[int(day[i])]
    for i in range(len(open)):
        if close[i]<open[i]:
            close[i]+=24
    for i in range(len(open)):
        serve+=[close[i]-open[i]]
    return serve

fri1=serve(fri)
mon1=serve(mon)
tue1=serve(tue)
wed1=serve(wed)
thu1=serve(thu)
sat1=serve(sat)
sun1=serve(sun)

df['Fridayophour']=fri1
df['Mondayophour']=mon1
df['Tuesdayophour']=tue1
df['Wednesdayophour']=wed1
df['Thursdayophour']=thu1
df['Saturdayophour']=sat1
df['Sundayophour']=sun1
df = df.iloc[:, 7:]

df.to_csv('Business_ophour.csv', index = False)

In [107]:
df = pd.read_csv('business_restaurant_orgin.csv')

In [108]:
df2 = pd.read_csv('Business_ophour.csv')

In [109]:
df = pd.merge(df, df2, left_index=True, right_index=True, how='left')

In [110]:
# drop columns with more than 1000 na

In [111]:
column0 = df.columns
df.isnull().sum()[df.isnull().sum()>1000]
column1 = df.isnull().sum()[df.isnull().sum()>1000].index
column = [val for val in column0 if val not in column1]
df = df[column]
df.shape

(59371, 111)

In [112]:
# drop working hours

In [113]:
df.drop(['hours_Friday', 'hours_Monday', 'hours_Saturday',
        'hours_Sunday', 'hours_Thursday', 'hours_Tuesday', 
        'hours_Wednesday'], axis = 1, inplace = True)
df.shape

(59371, 104)

In [115]:
# We first delete rows with more than 10 missing values, and fill the nan with the mode. 
# However, we found that way will bring noise to the datasets, and finally we delete rows 
# with all missing values

In [116]:
missing_index = list(df.isnull().sum(axis = 1)[df.isnull().sum(axis = 1)>0].index)
df = df.drop(index = missing_index)
df.shape

(55457, 104)

In [117]:
# summarize output categories

In [118]:
print('High quality count: %s'% len(df[df['stars']>=4]))
print('Low quality count: %s'% len(df[df['stars']<4]))
print('High quality proportion: %.8f' % float(len(df[df['stars']>=4])/(len(df[df['stars']<4])+len(df[df['stars']>=4]))))

High quality count: 21452
Low quality count: 34005
High quality proportion: 0.38682222


In [119]:
# convert stars format in dataset: 0 for low quality; 1 for high

In [120]:
df.loc[:, 'stars'] = (df['stars']>=4)*1
df['stars'].value_counts()

0    34005
1    21452
Name: stars, dtype: int64

In [121]:
# convert true/false to 1/0

In [122]:
df = 1*df

In [123]:
df.shape

(55457, 104)

In [125]:
df.to_csv('Business_restaurant_complete.csv', index = False)

### Now reduce some obviously irrelevent columns

In [16]:
df = pd.read_csv('Business_restaurant_complete.csv')

In [17]:
# reduce some obviously irrelevent or not-suitable for model-training columns based on research
# online
drop_columns = ['address',
                'attributes_AcceptsInsurance',
                'attributes_ByAppointmentOnly',
                'attributes_HairSpecializesIn_africanamerican',
                'attributes_HairSpecializesIn_asian',
                'attributes_HairSpecializesIn_coloring',
                'attributes_HairSpecializesIn_curly',
                'attributes_HairSpecializesIn_extensions',
                'attributes_HairSpecializesIn_kids',
                'attributes_HairSpecializesIn_perms',
                'attributes_HairSpecializesIn_straightperms',
                'is_open',
                'name',
                'postal_code',
                'state',
                'categories',
                'city']

In [18]:
df.drop(drop_columns, axis = 1, inplace = True)
df.shape

(55457, 87)

In [19]:
# rename column names
features = ['AgesAllowed',
            'Alcohol',
            'Ambience_casual',
            'Ambience_classy',
            'Ambience_divey',
            'Ambience_hipster',
            'Ambience_intimate',
            'Ambience_romantic',
            'Ambience_touristy',
            'Ambience_trendy',
            'Ambience_upscale',
            'BYOB',
            'BYOBCorkage',
            'BestNights_friday',
            'BestNights_monday',
            'BestNights_saturday',
            'BestNights_sunday',
            'BestNights_thursday',
            'BestNights_tuesday',
            'BestNights_wednesday',
            'BikeParking',
            'AcceptsBitcoin',
            'AcceptsCreditCards',
            'Parking_garage',
            'Parking_lot',
            'Parking_street',
            'Parking_valet',
            'Parking_validated',
            'Caters',
            'CoatCheck',
            'Corkage',
            'DietaryRestrictions_dairy-free',
            'DietaryRestrictions_gluten-free',
            'DietaryRestrictions_halal',
            'DietaryRestrictions_kosher',
            'DietaryRestrictions_soy-free',
            'DietaryRestrictions_vegan',
            'DietaryRestrictions_vegetarian',
            'DogsAllowed',
            'DriveThru',
            'GoodForDancing',
            'GoodForKids',
            'GoodForBreakfast',
            'GoodForBrunch',
            'GoodForDessert',
            'GoodForDinner',
            'GoodForLatenight',
            'GoodForLunch',
            'HappyHour',
            'HasTV',
            'BGM',
            'DJ',
            'Jukebox',
            'Karaoke',
            'Live',
            'NoMusic',
            'Video',
            'Noisy',
            'Outdoor',
            'Attire',
            'Delivery',
            'Group',
            'Price',
            'Reservation',
            'TableService',
            'TakeOut',
            'Smoking',
            'Wheelchair',
            'Wifi',
            'business_id', 
            'latitude', 
            'longitude',
            'review_count',
            'stars',
            'Breakfast',
            'Brunch',
            'Lunch',
            'Dinner',
            'Earlynight',
            'Latenight',
            'Fridayophour',
            'Mondayophour',
            'Tuesdayophour',
            'Wednesdayophour',
            'Thursdayophour',
            'Saturdayophour',
            'Sundayophour']

In [20]:
df.columns = features

In [21]:
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)

In [22]:
class_mapping = {'quiet':0, 'average':1, 'loud': 2, 'very_loud': 3}
df['Noisy'] = df['Noisy'].map(class_mapping)

In [23]:
df['Noisy'].value_counts()

1    36276
0    12582
2     4776
3     1823
Name: Noisy, dtype: int64

In [24]:
business_id = df['business_id']

In [25]:
df = pd.get_dummies(df.drop('business_id', axis = 1))

In [26]:
df['business_id'] = business_id

In [27]:
df.shape

(55457, 100)

In [84]:
df.to_csv('Business_final.csv', index = False)

### 3. Processing Reviews

In [1]:
from pandas.io.json import json_normalize  
import pandas as pd  
import json  

In [6]:
df = pd.read_csv('Business_final.csv')
df.shape

(55457, 100)

In [7]:
business_id = df['business_id'].to_frame()

In [3]:
yelp_reader = pd.read_json('review.json', lines=True, chunksize=100000)

In [4]:
# select all the reviews matched with our business data and the 'useful' column 
# should be more than 2 which indicates the reviews are 
# reliable (our assumption)

In [5]:
chunks = list()
i = 0
for chunk in yelp_reader:
    chunks.append(chunk)

In [8]:
yelp_reviews = pd.concat(chunks)

In [281]:
yelp_reviews = yelp_reviews[['business_id', 'text', 'useful', 'stars']]

In [282]:
# select comments with usefulness larger than 2, which indicates a relatively fair evaluation of a restaurant
yelp_reviews = yelp_reviews[yelp_reviews['useful'] >2 ]

In [283]:
yelp_reviews.shape

(1015774, 4)

In [284]:
restaurant_reviews = business_id.merge(yelp_reviews, how='left', left_on='business_id', right_on='business_id')

In [285]:
restaurant_reviews.dropna(axis=0, how='any', inplace=True)

In [286]:
restaurant_reviews.shape

(480562, 4)

In [287]:
len(restaurant_reviews['business_id'].unique())

42270

In [288]:
len(business_id)

55457

In [289]:
restaurant_reviews.to_csv('restaurant_reviews.csv', index = False)